In [ ]:
import time
import os
import json
import asyncio
import random
from typing import List
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from helpers.selenium_management import start_driver, open_link, get_wait_element, get_wait_elements, close_driver, get_link_elements, get_links
from helpers.utils import extract_original_link, parse_activity_data, validate_instagram_url #, validate_and_extract_domain
from helpers.excel import PyXLWriter

import config
from robot import auth, account_get_post_links, account_send_comment

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

import re
import string
from urllib.parse import urlparse
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from ml_models.custom_transformers import TextCleaner, DomainExtractor

In [ ]:
# Создание DataFrame
file_path = 'data/modеls/train_data/merged_table.xlsx'
df = pd.read_excel(file_path)

df.head()

In [ ]:
##################################
# Шаг 1: Предобработка текста
##################################

# Создание DataFrame
file_path = 'data/modеls/train_data/merged_table.xlsx'
df = pd.read_excel(file_path)

# Замена пустых значений
df.fillna(
    {   
        'Предсказанный тип аккаунта': 'OTHER',
        'Описание страницы': '',
        'Ссылки из описания': '', 
        'Ссылки из контактов': '',
        'Кол-во постов': 0
    }, 
    inplace=True
)

# 1 шаг: Замена имени столбца
# 2 шаг: Удаление всех строк с типом "OTHER"
df.rename(columns={"Предсказанный тип аккаунта": "Тип аккаунта"}, inplace=True)
df = df[df["Тип аккаунта"] != "OTHER"]

# Заменяем значения столбца на 1, если не NaN, и на 0, если NaN
# 1 шаг - замена на True и False
# 2 шаг - замена на 1 и 0
df['Почта существует'] = df['Найденная почта'].notna().astype(int)


##################################
# Шаг 2: Делим на X и y + разделяем данные на обучающую и тестовую выборки
##################################
X = df[[
    "Описание страницы",  # text
    "Ссылки из описания",  # text
    "Ссылки из контактов",  # text
    "Почта существует",  # 1/0
    "Кол-во постов",  # num
]]
y = df["Тип аккаунта"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

##################################
# Шаг 3: Настраиваем векторизацию и классификацию + cобираем Pipeline 
##################################

# Создание ColumnTransformer для обработки каждого столбца
preprocessor = ColumnTransformer(
    transformers=[
        (
            'desc_tfidf',  # Для текстовых значений 
            Pipeline([
                ('text_cleaner', TextCleaner()),
                ('tfidf', TfidfVectorizer(stop_words='english',  # Стоп-слова
                                          ngram_range=(1, 3),  # Использование триграмм
                                          max_features=1250,  # Увеличение числа признаков
                                          sublinear_tf=True)  # Использование логарифмического масштаба для частоты терминов
                ),
            ]),
            'Описание страницы'
            
        ),
        (
            'desc_links_tfidf', 
            Pipeline([
                ('extract_domains', DomainExtractor()),  # Кастомный экстрактор для ссылок
                ('tfidf', TfidfVectorizer())  # Для текстовых значений
            ]), 
            'Ссылки из описания'
        ),
        (
            'contact_links_tfidf', 
            Pipeline([
                ('extract_domains', DomainExtractor()),  # Кастомный экстрактор для ссылок
                ('tfidf', TfidfVectorizer())  # Для текстовых значений
            ]),
            'Ссылки из контактов'
        ),
        (
            'binary', 
            'passthrough',  # Пропускает значения без изменений
            ['Почта существует']
        ),
        (
            'num_scaler', 
            StandardScaler(),  # Для числовых значений
            ['Кол-во постов']
        ),
    ],
    remainder='drop'  # Удаляет остальные столбцы, если они есть
)


##################################
# Шаг 4: Собираем Pipeline
##################################

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline


# Использование методов ресэмплинга
pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),  # Увеличение числа примеров для редких классов с помощью методов, таких как SMOTE.
    ('clf', LogisticRegression(class_weight='balanced', max_iter=1000))  # 0.77
])

# ##################################
# # Шаг 5: Обучение и оценка
# ##################################

# Обучаем
pipeline.fit(X_train, y_train)

# Предсказываем
y_pred = pipeline.predict(X_test)

# Смотрим отчёт
print(classification_report(y_test, y_pred, zero_division=0))

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0))

In [18]:
import joblib

##################################
# Шаг 6: Обучение модели на всех имеющихся данных
##################################

pipeline.fit(X, y)

##################################
# Шаг 7: Сохранение модели
##################################

model_path = 'data/modеls/account_type.pkl'
joblib.dump(pipeline, model_path)
print(f"Модель сохранена в файле {model_path}")


##################################
# Шаг 8: Загрузка модели
##################################

loaded_model = joblib.load(model_path)
print(f"Модель загружена из файла {model_path}")


##################################
# Шаг 9: Использование модели
##################################

new_data = pd.DataFrame({
    'Описание страницы': [
        '''Colt Blumenthal
mixedbycolt
🚀 #1 Billboard Engineer, 💿 Multi-Platinum
💻Engineered for 100+ Major Artists
🏅 SAE Institute Alumni Hall of Fame
🔊Premium Mixing & Mastering Service
linktr.ee/mixedbycolt'''
    ],
    'Ссылки из описания': [
        '''https://linktr.ee/mixedbycolt?fbclid=PAZXh0bgNhZW0CMTEAAaYAv9ZUzP17l1o8wVsSVjvTAzUJT9CppNIKLuP0FENyTVKEOAH3OmmBnCA_aem_FRFJBiIdaHkZ9aYHEaCLHQ
https://www.instagram.com/explore/tags/1/
https://www.threads.net/@mixedbycolt?xmt=AQGzUA43HPJ5PxWfBGKXf29C7EUpu6r6IVFcdtq1jbBJa8Y'''
    ],
    'Ссылки из контактов': [
        ''
        # 'https://soundcloud.com/playlist/ddw31e\nhttps://youtube.com/watch?v=12345'
    ],
    'Почта существует': [0],
    'Кол-во постов': [90]
})

preds = loaded_model.predict(new_data)
print("Предсказанные классы:", preds)

proba = loaded_model.predict_proba(new_data)
print("Вероятности классов:", proba)

Модель сохранена в файле data/modеls/account_type.pkl
Модель загружена из файла data/modеls/account_type.pkl
Предсказанные классы: ['BEATMAKER']
Вероятности классов: [[0.03556533 0.88062202 0.01906485 0.04723903 0.01750876]]


In [21]:
preds[0]

'BEATMAKER'

In [17]:
# Подсчет каждого типа аккаунта
print(df['Тип аккаунта'].value_counts())


Тип аккаунта
ARTIST       358
COMMUNITY    179
MARKET        83
LABEL         37
BEATMAKER     19
Name: count, dtype: int64


In [ ]:
# precision (Точность) - какая доля действительно относится к этому классу?
# recall (Полнота) - какая доля была предсказана как этот класс?
# f1-score (Среднее гармоническое между точностью и полнотой)
# support (Число реальных образцов)

# accuracy: Общая доля правильных предсказаний по всем классам.

# avg: Средние значения precision, recall и f1-score по всем классам (без учёта их пропорций). 
# То есть каждый класс “взвешен” одинаково, даже если в выборке один класс встречается гораздо чаще другого.

# weighted avg: Взвешенное среднее тех же метрик, где вес — это support (количество объектов каждого класса). 
# Таким образом, частые классы влияют на метрики сильнее, чем редкие.


In [ ]:
# Фильтруем только строки, где "Предсказанный тип аккаунта" == "ARTIST"
df_artist = df[df["Предсказанный тип аккаунта"] == "ARTIST"]

# Считаем самые популярные хэштеги среди этих строк
hashtags_count = df_artist["Хэштег, по которому найден аккаунт"].value_counts()

# Смотрим результат (топ 10, например)
print(hashtags_count.head(10))


In [ ]:
duplicates_mask = df.duplicated()
print("Число дубликатов:", duplicates_mask.sum())

### Объединение данных с обработанными таблицами

In [1]:
import os
import pandas as pd
import numpy as np

from helpers.excel import write_excel

from database.orm import async_session, get_all_accounts


In [ ]:
# Папка с файлами .ods
folder_path = 'data/modеls/tables'

# Получаем список файлов .ods в папке
files = [f for f in os.listdir(folder_path) if f.endswith('.ods')]

# Создаем пустой DataFrame для объединенных данных
combined_df = pd.DataFrame()

for file in files:
    # Загружаем данные из каждого файла
    file_path = os.path.join(folder_path, file)
    data = pd.read_excel(file_path, engine='odf')  # Используем 'odf' для .ods файлов
    
    # Очистка данных
    data['Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER'] = data['Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER'].replace(np.nan, 'OTHER')

    # Объединяем данные по названию столбцов
    combined_df = pd.concat([combined_df, data], ignore_index=True)

async def update_account_types(async_session, data: pd.DataFrame):
    async with async_session() as session:
        # Получаем все аккаунты из базы данных
        accounts = await get_all_accounts(async_session)

        # Создаем словарь для быстрого поиска аккаунтов по ссылке
        account_dict = {account.link: account for account in accounts}

        # Обновляем типы аккаунтов на основе данных из DataFrame
        for _, row in data.iterrows():
            link = row.get('Ссылка на аккаунт').strip()
            account_type = row.get('Тип аккаунта (записывается всегда). Значения: ARTIST, BEATMAKER, LABEL, MARKET, COMMUNITY, OTHER').strip()
            # print(f'link: {link}')
            # print(f'account_type: {account_type}')

            if link in account_dict and account_type:
                account = account_dict[link]
                account.account_type = account_type
                # print(f'account: {account.link}')
                # print(f'account_type: {account.account_type}')
                session.add(account)
        # Сохраняем изменения в базе данных
        await session.commit()
        
await update_account_types(async_session, combined_df)

In [ ]:
accounts = await get_all_accounts(async_session)
write_excel(accounts, out_path='data/modеls/row_data/merged_table.xlsx')